In [19]:
import nltk
from nltk.chat.util import Chat  , reflections

In [20]:
reflections

{'i am': 'you are',
 'i was': 'you were',
 'i': 'you',
 "i'm": 'you are',
 "i'd": 'you would',
 "i've": 'you have',
 "i'll": 'you will',
 'my': 'your',
 'you are': 'I am',
 'you were': 'I was',
 "you've": 'I have',
 "you'll": 'I will',
 'your': 'my',
 'yours': 'mine',
 'you': 'me',
 'me': 'you'}

In [21]:
print(Chat)

<class 'nltk.chat.util.Chat'>


In [22]:
set_pairs = [
    [
        r"my name is (.*)",
        ["hello %1 , How are you today?",]
    ],
    [
        r"hi|hey|hello",
        ["Hello" , 'Hey there',]
    ],
    [
        r"what is your name ?",
        ["You can call my a chatbot ?",]
    ],
    [
        r"How are you ?",
        ["I am fine , thank you ! How can i help you ?",]
    ],
    [
        r"I am fine , thank you",
        ["Great to hear that , How can i help you ?",]
    ],
    [
        r"How can i help you ?",
        ["i am looking for online guides and courses to learn data science , can you suggest some thing to me ?",]
    ],
    [
        r"i'm (.*) ",
        ["That's great to hear" , "how can i help you ? :)",]
    ],
    [
        r"i am looking for online guides and courses to learn data science , can you suggest some thing to me ?",
        ["elzero channel is a great option  to learn data science , you can check his channel"]
    ],
    [
        r"thank for the suggestion . do they have greate authors and instructors",
        ["yes , they have the world class best authors , that is their strength :)",]
    ],
    [
        r"(.*) thanks you so much , that was helpful",
        ["i'm happy to help" , "No problem , you are welcome",]
    ],
    [
        r"quit|end|thanks|goodbye|bye|see you soon",
        ["Bye , take care. see you soon :) ", "it was nice talking to you . see you soon :)"]
    ],
    [
        r"can you help me",
        ["yes , sure انت تامر ياسيد الناس"]
    ],
    [
        r"do you love me",
        ['sure , i love you prince']
    ],
]

In [23]:
chat = Chat(set_pairs , reflections)
print(chat)


In [24]:
chat.converse()

Hey there
hello khaled , How are you today?
hello khaled , How are you today?
yes , sure انت تامر ياسيد الناس
elzero channel is a great option  to learn data science , you can check his channel
elzero channel is a great option  to learn data science , you can check his channel
yes , they have the world class best authors , that is their strength :)
yes , they have the world class best authors , that is their strength :)
it was nice talking to you . see you soon :)


In [25]:
import nltk
import numpy as np
import random
import string
import bs4 as bs
import requests
import re
import warnings

warnings.filterwarnings = False

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\khaledtarek\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\khaledtarek\AppData\Roaming\nltk_data...


True

In [27]:
r = requests.get('https://en.wikipedia.org/wiki/Cuisine')
print(r)
raw_html = r.text

<Response [200]>


In [28]:
corpus_html = bs.BeautifulSoup(raw_html)
courps_paras = corpus_html.find_all('p')
courps_txt = ''

for paras in courps_paras:
    courps_txt += paras.text

courps_txt = courps_txt.lower()

In [29]:
print(courps_txt)


a cuisine  is a style of cooking characterized by distinctive ingredients, techniques and dishes, and usually associated with a specific culture or geographic region. regional food preparation techniques, customs, and ingredients  combine to enable dishes unique to a region.[1]
used in english since the late 18th century, the word cuisine – meaning manner or style of cooking – is borrowed from the french for "style of cooking," as originally derived from latin coquere "to cook".[2]
a cuisine is partly determined by ingredients that are available locally or through trade. regional ingredients are developed and commonly contribute to a regional or national cuisine, such as japanese rice in japanese cuisine.
religious food laws can also exercise an influence on cuisine, such as indian cuisine and hinduism that is mainly lacto-vegetarian (avoiding meat and eggs) due to sacred animal worship. sikhism in punjabi cuisine, buddhism in east asian cuisine, christianity in european cuisine,[3][4

In [30]:
courps_txt = re.sub(r'\[[0-9]*\]', ' ' , courps_txt)
courps_txt = re.sub(r'\s+' , ' ' , courps_txt)

In [31]:
courps_sentences = nltk.sent_tokenize(courps_txt)
courps_wwords = nltk.word_tokenize(courps_txt)

# courps_wwords
# courps_sentences

In [32]:
greeting_inputs = ('hey' , 'good morning' , 'good evening' , 'morning' , 'evening' , 'hi' , 'whatsup')
greeting_responses = ['hey' , 'hey how are you' , 'Hello , how you doing' , 'Hello' , 'Welcome , iam good']

def choise_response(greeting):
    for token in greeting.split():
        if token.lower() in greeting_inputs:
            return random.choice(greeting_responses)

In [33]:
lemetizer = nltk.stem.WordNetLemmatizer()

def lemmtize_courps(tokens):
    return [lemetizer.lemmatize(token) for token in tokens]

remove_puncet  = dict((ord(punctuation) , None) for punctuation in string.punctuation)

def get_text(document):
    return lemmtize_courps(nltk.word_tokenize(document.lower().translate(remove_puncet)))

In [34]:
def responed(user_input):
    bot_response = ''
    courps_sentences.append(user_input)

    word_vectorize = TfidfVectorizer(tokenizer=get_text , stop_words='english')
    corpus_word_vector = word_vectorize.fit_transform(courps_sentences)
    
    cos_sim_vector = cosine_similarity(corpus_word_vector[-1] , corpus_word_vector)
    similitry_response_idx = cos_sim_vector.argsort()[0][-2]
    
    matched_vector = cos_sim_vector.flatten()
    matched_vector.sort()
    
    vector_matched = matched_vector[-2]
    
    if vector_matched == 0:
        bot_response  = bot_response + "I am sorry  , what is it , again?"
        return bot_response
    else:
        bot_response = bot_response + courps_sentences[similitry_response_idx]
        return bot_response

In [35]:
chat = True
print('Hello , what do you want to learn about Cuisines today ?')
while(chat == True):
    user_quary = input()
    user_quary = user_quary.lower()
    if user_quary != 'quit':
        if user_quary == 'thanks' or user_quary == ' thank you' :
            chat = False
            print('CuisineBot: You are Welcome! :)')
        else:
            if choise_response(user_quary) != None:
                print('CuisineBot: ' + choise_response(user_quary))
            else:
                print('CuisineBot: ' , end='')
                print(responed(user_quary))
                courps_sentences.remove(user_quary)
    else:
        chat = False
        print('CuisineBot: Good Bye! :)')

Hello , what do you want to learn about Cuisines today ?
CuisineBot: hey how are you
CuisineBot: african cuisines use a combination of locally available fruits, cereals and vegetables, as well as milk and meat products.
CuisineBot: a global cuisine is a cuisine that is practiced around the world, and can be categorized according to the common use of major foodstuffs, including grains, produce and cooking fats.
CuisineBot: regional ingredients are developed and commonly contribute to a regional or national cuisine, such as japanese rice in japanese cuisine.
CuisineBot: spices at central market in agadir, morocco due to asia's vast size and extremely diverse geography and demographics, asian cuisines are many and varied, and include east asian cuisine, south asian cuisine, southeast asian cuisine, central asian cuisine and west asian cuisine.
CuisineBot: Good Bye! :)
